# Most Valuable Player (MVP) Analysis


This notebook provides an analysis to determine the Most Valuable Player (MVP) from various soccer leagues. 
The MVP is calculated based on a combination of positive impact metrics and negative impact metrics.


## Data Loading

In [30]:
import pandas as pd
player_stats_data = pd.read_csv('2022-2023 Football Player Stats.csv', encoding='ISO-8859-1', delimiter=';')

In [67]:
# set a minimum number of minutes played to be considered 
player_stats_data = player_stats_data[player_stats_data['Min'] >= 650]

## MVP Calculation

In [68]:
# Define weights for different metrics
positive_weights = {
    'Derived Shot on Target Percentage': 1.0,
    'Goals': 3.0,
    'PasAss': 0.1,
    'Pas3rd': 0.15,
    'PasProg': 0.15,
    'Sw': 0.15,
    'ScaDrib': 0.2,
    'ScaSh': 0.3,
    'GCA': 0.75,
    'PasMedCmp': 0.12,
    'PasLonCmp': 0.17,
    'Assists': 1.5,
    'PasAss': 0.4,
    'PPA': 0.25,
    'TB' : 0.25,
    'ScaPassLive': 0.3,
    'ScaFld': 0.05,
    'GcaDrib': 0.25
}



In [69]:
# Negative Weights
negative_weights = {
    'Off': -0.3,
    'CrdY': -0.5,
    'CrdR': -1.0,
    'Fls': -0.2,
    'PasBlocks': -0.0
}

In [70]:
# Adjust weights based on player position
position_adjusted_weights = {
    'Goals': {'DF': 1.2, 'MF': 1.1, 'FW': 1.0},
    'Int': {'FW': 1.2, 'MF': 1.1, 'DF': 1.0}  # Assuming 'Int' represents interceptions
}

In [71]:
# Update the existing weights with the position-adjusted weights
for metric, weight_dict in position_adjusted_weights.items():
    for position, weight in weight_dict.items():
        if metric in positive_weights:
            positive_weights[metric] *= weight

In [72]:
# Make sure we have a deep copy of filtered_data_half_season to work on
filtered_data_updated = filtered_data_half_season.copy()


In [73]:
# Compute the positive and negative impact scores with adjusted weights
for metric, weight in positive_weights.items():
    filtered_data_updated[metric + ' Weighted'] = filtered_data_updated[metric] * weight

for metric, weight in negative_weights.items():
    filtered_data_updated[metric + ' Weighted'] = filtered_data_updated[metric] * weight

filtered_data_updated['Positive Score'] = filtered_data_updated[[metric + ' Weighted' for metric in positive_weights.keys()]].sum(axis=1)
filtered_data_updated['Negative Score'] = filtered_data_updated[[metric + ' Weighted' for metric in negative_weights.keys()]].sum(axis=1)

In [74]:
# Adjust weights based on player position
position_adjusted_weights = {
    'Goals': {'DF': 1.2, 'MF': 1.1, 'FW': 1.0},
    'Int': {'FW': 1.2, 'MF': 1.1, 'DF': 1.0}  # Assuming 'Int' represents interceptions
}

In [75]:
# Compute the overall MVP score
filtered_data_updated['Adjusted MVP Score'] = filtered_data_updated['Positive Score'] + filtered_data_updated['Negative Score']

In [76]:
# Define the list of positions
positions = ['DF', 'MF', 'FW']

In [77]:
# Function to retrieve top 3 players for each position in each league
def top_3_by_position(league_data, position):
    return league_data[league_data['Pos'].str.contains(position)].nlargest(3, 'Adjusted MVP Score')[['Player', 'Adjusted MVP Score']]

In [78]:
# Collect the results
results_corrected = {}
for league in filtered_data_updated['Comp'].unique():
    league_data = filtered_data_updated[filtered_data_updated['Comp'] == league]
    results_corrected[league] = {position: top_3_by_position(league_data, position) for position in positions}

In [79]:
# Display the results in a formatted manner
formatted_results_corrected = ""
for league, positions_data in results_corrected.items():
    formatted_results_corrected += f"### {league}\n\n"
    for position, data in positions_data.items():
        formatted_results_corrected += f"**{position}**\n"
        formatted_results_corrected += data.to_string(index=False) + "\n\n"

print(formatted_results_corrected)

### Premier League

**DF**
          Player  Adjusted MVP Score
     Jonny Evans          107.259200
Cheikhou Kouyaté          105.730900
      Phil Foden           84.377669

**MF**
            Player  Adjusted MVP Score
  Cheikhou Kouyaté            105.7309
Idrissa Gana Gueye            104.1346
   Roberto Firmino             94.1677

**FW**
         Player  Adjusted MVP Score
 Erling Haaland          152.576871
     Harry Kane          116.071242
Roberto Firmino           94.167700

### Ligue 1

**DF**
      Player  Adjusted MVP Score
Lucas Perrin            104.3185
 Nuno Mendes             89.2300
 Juan Bernat             88.0853

**MF**
        Player  Adjusted MVP Score
        Neymar          107.289398
Martin Terrier           98.557900
  Lionel Messi           93.590466

**FW**
           Player  Adjusted MVP Score
Wissam Ben Yedder          123.447946
           Neymar          107.289398
    Kylian Mbappé          106.228300

### Serie A

**DF**
      Player  Adjusted MVP 